In [1]:
from numpy import average
import os
import graphlab as gl
import numpy as np
import gensim
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

gl.canvas.set_target('ipynb')

[INFO] This non-commercial license of GraphLab Create is assigned to marvin.bertin@gmail.comand will expire on September 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-900 - Server binary: /Users/marvinbertin/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444149317.log
[INFO] GraphLab Server Version: 1.6.1


In [2]:
traindata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv"

In [3]:
train_data = gl.SFrame.read_csv(traindata_path,header=True, delimiter='\t',quote_char='"',
                                column_type_hints = {'id':str, 'sentiment' : int, 'review':str } )

PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.484371 secs.
PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 25000 lines in 0.854796 secs.


In [6]:
unlabeledtraindata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/UnlabeledData/unlabeledTrainData.tsv"
unlabeled_train_data = gl.SFrame.read_csv(unlabeledtraindata_path,header=True, delimiter='\t',quote_char='"',
                                column_type_hints = {'id':str, 'review':str } )

PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/UnlabeledData/unlabeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.716355 secs.
PROGRESS: Read 38895 lines. Lines per second: 28436.5
PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/UnlabeledData/unlabeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 50000 lines in 1.56339 secs.


In [7]:
unlabeled_train_data.show()

In [8]:
train_data.show()

In [5]:
train_data.head()

id,sentiment,review
5814_8,1,With all this stuff goingdown at the moment with ...
2381_9,1,"""The Classic War of theWorlds"" by Timothy Hines ..."
7759_3,0,The film starts with amanager (Nicholas Bell) ...
3630_4,0,It must be assumed thatthose who praised this ...
9495_8,1,Superbly trashy andwondrously unpretentious ...
8196_8,1,I dont know why peoplethink this is such a bad ...
7166_2,0,"This movie could havebeen very good, but c ..."
10633_1,0,I watched this video at afriend's house. I'm glad ...
319_1,0,"A friend of mine boughtthis film for £1, and ..."
8713_10,1,<br /><br />This movie isfull of references. Like ...


In [6]:
train_data['review'].head(1)

dtype: str
Rows: 1
["With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit

In [9]:
BASE_DIR = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn"

In [8]:
os.mkdir("%s/txt" % BASE_DIR)
train_data.apply(lambda r: file("%s/txt/%s.txt" % (BASE_DIR, r["id"]),"w").write(r['review'])).__materialize__()

In [10]:
unlabeled_train_data.apply(lambda r: file("%s/txt/%s.txt" % (BASE_DIR, r["id"]),"w").write(r['review'])).__materialize__()

In [11]:
class TrainSentences(object):
    """
    Iterator class that returns Sentences from texts files in a input directory
    """
    RE_WIHTE_SPACES = re.compile("\s+")
    STOP_WORDS = set(stopwords.words("english"))
    def __init__(self, dirname):
        """
        Initialize a TrainSentences object with a input directory that contains text files for training
        :param dirname: directory name which contains the text files        
        """
        self.dirname = dirname

    def __iter__(self):
        """
        Sentences iterator that return sentences parsed from files in the input directory.
        Each sentences is returned as list of words
        """
        #First iterate  on all files in the input directory
        for fname in os.listdir(self.dirname):
            # read line from file (Without reading the entire file)
            for line in file(os.path.join(self.dirname, fname), "rb"):
                # split the read line into sentences using NLTK
                for s in txt2sentences(line, is_html=True):
                    # split the sentence into words using regex
                    w =txt2words(s, lower=True, is_html=False, remove_stop_words=False,
                                                 remove_none_english_chars=True)
                    #skip short sentences with less than 3 words
                    if len(w) < 3:
                        continue
                    yield w

def txt2sentences(txt, is_html=False, remove_none_english_chars=True):
    """
    Split the English text into sentences using NLTK
    :param txt: input text.
    :param is_html: If True thenremove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :return: string in which each line consists of single sentence from the original input text.
    :rtype: str
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # split text into sentences using nltk packages
    for s in tokenizer.tokenize(txt):
        if remove_none_english_chars:
            #remove none English chars
            s = re.sub("[^a-zA-Z]", " ", s)
        yield s

In [12]:
def txt2words(txt, lower=True, is_html=False, remove_none_english_chars=True, remove_stop_words=True):
    """
    Split text into words list
    :param txt: the input text
    :param lower: if to make the  text to lowercase or not.
    :param is_html: If True then  remove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :param remove_stop_words: if True then remove stop words from text
    :return: words list create from the input text according to the input parameters.
    :rtype: list
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    if lower:
        txt = txt.lower()
    if remove_none_english_chars:
        txt = re.sub("[^a-zA-Z]", " ", txt)

    words = TrainSentences.RE_WIHTE_SPACES.split(txt.strip().lower())
    if remove_stop_words:
        #remove stop words from text
        words = [w for w in words if w not in TrainSentences.STOP_WORDS]
    return words

In [13]:
sentences = TrainSentences("%s/LabeledData" % BASE_DIR)
model = gensim.models.Word2Vec(sentences, size=300, workers=4, min_count=40, window=8)
model.save("%s/movie_fullreviews_300_c_40_w_8.word2vec" % BASE_DIR)

In [14]:
model.most_similar("horrible")

[(u'terrible', 0.7421640157699585),
 (u'awful', 0.5961873531341553),
 (u'atrocious', 0.5896052122116089),
 (u'horrendous', 0.5849360227584839),
 (u'dreadful', 0.5772282481193542),
 (u'lousy', 0.5752981901168823),
 (u'horrid', 0.5428860187530518),
 (u'pathetic', 0.5386751294136047),
 (u'ridiculous', 0.5067158937454224),
 (u'stupid', 0.5062626600265503)]

In [15]:
model.most_similar("terrific")

[(u'superb', 0.6797044277191162),
 (u'wonderful', 0.6392487287521362),
 (u'fantastic', 0.6221407651901245),
 (u'marvellous', 0.5957516431808472),
 (u'brilliant', 0.5918124318122864),
 (u'fine', 0.5870006680488586),
 (u'stellar', 0.5849796533584595),
 (u'fabulous', 0.5810800194740295),
 (u'impeccable', 0.5685657262802124),
 (u'solid', 0.566067099571228)]

In [16]:
train_data.num_rows

<bound method SFrame.num_rows of Columns:
	id	str
	sentiment	int
	review	str

Rows: 25000

Data:
+---------+-----------+-------------------------------+
|    id   | sentiment |             review            |
+---------+-----------+-------------------------------+
|  5814_8 |     1     | With all this stuff going ... |
|  2381_9 |     1     | "The Classic War of the Wo... |
|  7759_3 |     0     | The film starts with a man... |
|  3630_4 |     0     | It must be assumed that th... |
|  9495_8 |     1     | Superbly trashy and wondro... |
|  8196_8 |     1     | I dont know why people thi... |
|  7166_2 |     0     | This movie could have been... |
| 10633_1 |     0     | I watched this video at a ... |
|  319_1  |     0     | A friend of mine bought th... |
| 8713_10 |     1     | <br /><br />This movie is ... |
+---------+-----------+-------------------------------+
[25000 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n

In [17]:
class DeepTextAnalyzer(object):
    def __init__(self, word2vec_model):
        """
        Construct a DeepTextAnalyzer using the input Word2Vec model
        :param word2vec_model: a trained Word2Vec model
        """
        self._model = word2vec_model

    def txt2vectors(self,txt, is_html):
        """
        Convert input text into an iterator that returns the corresponding vector representation of each
        word in the text, if it exists in the Word2Vec model
        :param txt: input text
        :param is_html: if True, then extract the text from the input HTML
        :return: iterator of vectors created from the words in the text using the Word2Vec model.
        """
        words = txt2words(txt,is_html=is_html, lower=True, remove_none_english_chars=True)
        words = [w for w in words if w in self._model]
        if len(words) != 0:
            for w in words:
                yield self._model[w]


    def txt2avg_vector(self, txt, is_html):
        """
        Calculate the average vector representation of the input text
        :param txt: input text
        :param is_html: is the text is a HTML
        :return the average vector of the vector representations of the words in the text  
        """
        vectors = self.txt2vectors(txt,is_html=is_html)
        vectors_sum = next(vectors, None)
        if vectors_sum is None:
            return None
        count =1.0
        for v in vectors:
            count += 1
            vectors_sum = np.add(vectors_sum,v)
        
        #calculate the average vector and replace +infy and -inf with numeric values 
        avg_vector = np.nan_to_num(vectors_sum/count)
        return avg_vector

In [18]:
dt = DeepTextAnalyzer(model)
train_data['vectors'] = train_data['review'].apply(lambda p: dt.txt2avg_vector(p, is_html=True))
train_data['vectors'].head(1)

dtype: array
Rows: 1
[array('d', [0.0012614948209375143, 0.008466312661767006, -0.014087257906794548, -0.0016300550196319818, -0.007059338502585888, 0.04334469512104988, -0.01818360947072506, 0.00011644816549960524, 0.016906248405575752, -0.03242700174450874, 0.015863699838519096, 0.04014977067708969, -0.0028936213348060846, -0.006295966915786266, -0.040098886936903, -0.013835880905389786, 0.04316890612244606, -0.04829014837741852, 0.037902530282735825, 0.03207206353545189, 0.04304228723049164, -0.053176090121269226, 0.027713846415281296, 0.030210109427571297, 0.02624749019742012, 0.037598468363285065, 0.003321701195091009, 0.05034821853041649, -0.04806441813707352, -0.047715410590171814, -0.0026634447276592255, 0.013868541456758976, -0.007943634875118732, -0.00863218680024147, 0.003390579018741846, 0.054701801389455795, -0.015039627440273762, 0.013414383865892887, 0.00010467922402312979, 0.019721226766705513, -0.00816380139440298, 0.009082430973649025, 0.021328585222363472, -0.0041816

In [19]:
train_data.num_rows()

25000

In [20]:
train_data.head()

id,sentiment,review,vectors
5814_8,1,With all this stuff goingdown at the moment with ...,"[0.00126149482094,0.00846631266177, ..."
2381_9,1,"""The Classic War of theWorlds"" by Timothy Hines ...","[0.00941793154925,0.00266537535936, ..."
7759_3,0,The film starts with amanager (Nicholas Bell) ...,"[0.000344402680639,-0.0448061563075, ..."
3630_4,0,It must be assumed thatthose who praised this ...,"[0.0226462651044,-0.0204022880644, ..."
9495_8,1,Superbly trashy andwondrously unpretentious ...,"[0.0169358514249,-0.00542118074372, ..."
8196_8,1,I dont know why peoplethink this is such a bad ...,"[0.0193388033658,0.00649426970631, ..."
7166_2,0,"This movie could havebeen very good, but c ...","[0.0182176362723,-0.0564987808466, ..."
10633_1,0,I watched this video at afriend's house. I'm glad ...,"[-0.0137769971043,0.0515923947096, ..."
319_1,0,"A friend of mine boughtthis film for £1, and ...","[0.0589527860284,-0.0169450119138, ..."
8713_10,1,<br /><br />This movie isfull of references. Like ...,"[-0.0252398718148,0.0132357999682, ..."


In [26]:
train_subset, test_subset = train_data.random_split(0.8, seed=1)

In [28]:
cls = gl.classifier.create(train_subset, target="sentiment", features=["vectors"])
result1 = cls.evaluate(test_subset)
result1

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 18930
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Number of coefficients    : 301
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+---

{'accuracy': 0.8534072900158478, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  392  |
 |      1       |        0        |  348  |
 |      1       |        1        |  2216 |
 |      0       |        0        |  2092 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [31]:
cls = gl.classifier.neuralnet_classifier.create(train_subset, target="sentiment", features=["vectors"],
                                                metric=['accuracy', 'recall@2'],
                                                max_iterations=3)
result2 = cls.evaluate(test_subset)
result2

Using network:

### network layers ###
layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 2
layer[3]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Creating neuralnet using cpu
PROGRESS: Training with batch size = 100
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+
PROGRESS: | Iteration | Examples | Elapsed Time | Training-accuracy | Validation-accuracy | Training-recall@2 | Validation-recall@2 

{'accuracy': 0.4920760691165924, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 2
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  2484 |
 |      1       |        0        |  2564 |
 +--------------+-----------------+-------+
 [2 rows x 3 columns]}

In [32]:
cls = gl.classifier.random_forest_classifier.create(train_subset, target="sentiment", features=["vectors"])
result3 = cls.evaluate(test_subset)
result3

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Random forest classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 18879
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   7.592e-01   7.018e-01        1.10s
PROGRESS:      1   7.815e-01   7.409e-01        1.82s
PROGRESS:      2   7.943e-01   7.418e-01        2.54s
PROGRESS:      3   7.959e-01   7.418e-01        3.27s
PROGRESS:      4   7.986e-01   7.521e-01        4.03s
PROGRESS:      5   8.006e-01   7.577e-01        4.77s
PROGRESS:      6   8.058e-01   7.

{'accuracy': 0.7670364500792393, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  600  |
 |      1       |        1        |  1988 |
 |      1       |        0        |  576  |
 |      0       |        0        |  1884 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [33]:
cls = gl.classifier.boosted_trees_classifier.create(train_subset, target="sentiment", features=["vectors"])
result4 = cls.evaluate(test_subset)
result4

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 19002
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   7.668e-01   7.358e-01        1.17s
PROGRESS:      1   7.920e-01   7.432e-01        2.35s
PROGRESS:      2   8.158e-01   7.800e-01        3.54s
PROGRESS:      3   8.250e-01   7.832e-01        4.82s
PROGRESS:      4   8.365e-01   7.937e-01        6.17s
PROGRESS:      5   8.469e-01   7.853e-01        7.45s
PROGRESS:      6   8.530e-01   7.

{'accuracy': 0.8019017432646592, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  515  |
 |      1       |        1        |  2079 |
 |      1       |        0        |  485  |
 |      0       |        0        |  1969 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [21]:
cls = gl.classifier.create(train_data, target='sentiment', features=['vectors'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 23750
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Number of coefficients    : 301
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+---

In [22]:
#creating the test dataset
testdata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv"
test_data = gl.SFrame.read_csv(testdata_path,header=True, delimiter='\t',quote_char='"',
                               column_type_hints = {'id':str, 'review':str } )

PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.521591 secs.
PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv
PROGRESS: Parsing completed. Parsed 25000 lines in 0.852852 secs.


In [23]:
dt = DeepTextAnalyzer(model)
test_data['vectors'] = test_data['review'].apply(lambda p: dt.txt2avg_vector(p, is_html=True))
test_data['vectors'].head(1)

dtype: array
Rows: 1
[array('d', [-0.03128037229180336, -0.051567815244197845, 0.03276579827070236, 0.003264856757596135, 0.002818173263221979, 0.02964898757636547, 0.005326887127012014, 0.014321301132440567, -0.05812147632241249, -0.0457916297018528, 0.03189953789114952, 0.059900715947151184, 0.02898985520005226, -0.02272074483335018, -0.030947143211960793, 0.005725131835788488, 0.0141909159719944, -0.06646397709846497, 0.0056731486693024635, 0.021826529875397682, -0.007459764368832111, 0.006760365329682827, 0.010986651293933392, 0.023761263117194176, 0.10277367383241653, -0.004730912856757641, 0.06891278922557831, -0.024212660267949104, 0.015612373128533363, -0.018157178536057472, -0.07257542759180069, -0.012022624723613262, 0.04104430601000786, -0.09585385024547577, 0.017209835350513458, -0.03303062170743942, -0.046203937381505966, 0.0028699974063783884, -0.038716431707143784, 0.04082956165075302, -0.01897786371409893, -0.000754414068069309, 0.03310908004641533, 0.010073383338749409

In [24]:
test_data['sentiment'] = cls.classify(test_data)['class'].astype(int)
test_data.head()

id,review,vectors,sentiment
12311_10,Naturally in a film who'smain themes are of ...,"[-0.0312803722918,-0.0515678152442, ...",1
8348_2,This movie is a disasterwithin a disaster film. ...,"[-0.0244557447731,0.00596945919096, ...",0
5828_4,"All in all, this is amovie for kids. We sa ...","[-0.0136676458642,0.00491039827466, ...",0
7186_2,Afraid of the Dark leftme with the impression ...,"[-0.007137932349,-0.0341661460698, ...",0
12128_7,A very accurate depictionof small time mob life ...,"[-0.0194276254624,-0.0178465656936, ...",1
2913_8,"...as valuable as KingTut's tomb! (OK, maybe ...","[-0.0231488514692,-0.0022550271824, ...",1
4396_1,This has to be one of thebiggest misfires ...,"[-0.0100834686309,0.0197394713759, ...",0
395_2,"This is one of thosemovies I watched, and ...","[-0.0286349263042,0.0193572770804, ...",1
10616_1,The worst movie i've seenin years (and i've se ...,"[0.0473213978112,0.0812859684229, ...",0
9074_9,"Five medical students(Kevin Bacon, David ...","[-0.0326251126826,-0.017272606492, ...",0


In [25]:
#saving the prediction to a CSV for submission
test_data[['id', 'sentiment']].save("/Users/marvinbertin/Github/Graphlab/DeepTextpred3.csv", format="csv")
# pred1: score 0.865
# pred2: score 0.859
# pred3: score 0.864